<a href="https://colab.research.google.com/github/shahabas9/Deep_learning_basics/blob/main/functiional_model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle

In [2]:
!cp kaggle.json  ~/.kaggle/

In [3]:
!kaggle datasets download -d jangedoo/utkface-new

100% 331M/331M [00:16<00:00, 23.5MB/s]
100% 331M/331M [00:16<00:00, 21.2MB/s]


In [4]:
import zipfile

In [5]:
zip_ref=zipfile.ZipFile('/content/utkface-new.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
import os

In [9]:
age=[]
gender=[]
img=[]
for file in os.listdir("/content/utkface_aligned_cropped/UTKFace"):
  age.append(int(file.split("_")[0]))
  gender.append(int(file.split("_")[1]))
  img.append(file)

In [11]:
len(gender)

23708

In [12]:
import pandas as pd

In [14]:
data=pd.DataFrame({"age":age,"gender":gender,"file_name":img})

In [16]:
data.head()

,age,gender,file_name
0,11,1,11_1_4_20170109201611941.jpg.chip.jpg
1,73,0,73_0_1_20170117194721939.jpg.chip.jpg
2,27,1,27_1_0_20170117014038281.jpg.chip.jpg
3,21,0,21_0_1_20170115231442487.jpg.chip.jpg
4,50,0,50_0_0_20170117152322862.jpg.chip.jpg


In [17]:
train=data.sample(frac=1,random_state=42).iloc[:20000]

test=data.sample(frac=1,random_state=42).iloc[20000:]

In [18]:
train.shape

(20000, 3)

In [19]:
test.shape

(3708, 3)

In [20]:
from keras.preprocessing.image import ImageDataGenerator

In [21]:
train_generator=ImageDataGenerator(
    rotation_range=30,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=1/255
)



test_generator=ImageDataGenerator(

    rescale=1/255
)


In [23]:
train_datagen=train_generator.flow_from_dataframe(train,
                                                  directory="/content/utkface_aligned_cropped/UTKFace",
                                                  x_col="file_name",
                                                  y_col=["age","gender"],
                                                  target_sixe=(200,200),
                                                  class_mode="multi_output")

test_datagen=test_generator.flow_from_dataframe(test,
                                                directory="/content/utkface_aligned_cropped/UTKFace",
                                                x_col="file_name",
                                                y_col=["age","gender"],
                                                target_sixe=(200,200),
                                                class_mode="multi_output")

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [24]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [25]:
resnet=ResNet50(include_top=False,
         input_shape=(200,200,3))

94765736/94765736 [==============================] - 5s 0us/step


In [43]:
resnet.trainable=False

In [44]:
output=resnet.layers[-1].output

In [45]:
flatten=Flatten()(output)

In [46]:
dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512, activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [47]:
model=Model(inputs=resnet.input,outputs=[output1,output2])

In [48]:
model.compile(optimizer='adam',loss={"age":"mse","gender":'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'})

In [ ]:
model.fit(train_datagen,batch_size=32,epochs=2,validation_data=test_datagen)

In [50]:
train_datagen = train_generator.flow_from_dataframe(train,
                                                  directory="/content/utkface_aligned_cropped/UTKFace",
                                                  x_col="file_name",
                                                  y_col=["age", "gender"],
                                                  target_size=(200, 200),
                                                  class_mode="multi_output")

test_datagen = test_generator.flow_from_dataframe(test,
                                                directory="/content/utkface_aligned_cropped/UTKFace",
                                                x_col="file_name",
                                                y_col=["age", "gender"],
                                                target_size=(200, 200),
                                                class_mode="multi_output")

resnet.trainable = False
output = resnet.layers[-1].output
flatten = Flatten()(output)
dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512, activation='relu')(flatten)

dense3 = Dense(512, activation='relu')(dense1)
dense4 = Dense(512, activation='relu')(dense2)

output1 = Dense(1, activation='linear', name='age')(dense3)
output2 = Dense(1, activation='sigmoid', name='gender')(dense4)
model = Model(inputs=resnet.input, outputs=[output1, output2])

model.compile(optimizer='adam', loss={"age": "mean_squared_error", "gender": "binary_crossentropy"}, metrics={"age": "mae", "gender": "accuracy"})
model.fit(train_datagen, batch_size=32, epochs=2, validation_data=test_datagen)


Found 20000 validated image filenames.
Found 3708 validated image filenames.
Epoch 1/2
625/625 [==============================] - 256s 397ms/step - loss: 416.4502 - age_loss: 415.5307 - gender_loss: 0.9199 - age_mae: 15.7376 - gender_accuracy: 0.5096 - val_loss: 445.4510 - val_age_loss: 444.7587 - val_gender_loss: 0.6925 - val_age_mae: 15.4591 - val_gender_accuracy: 0.5232
Epoch 2/2
625/625 [==============================] - 234s 374ms/step - loss: 383.2423 - age_loss: 382.5494 - gender_loss: 0.6928 - age_mae: 15.0450 - gender_accuracy: 0.5228 - val_loss: 367.5234 - val_age_loss: 366.8311 - val_gender_loss: 0.6924 - val_age_mae: 15.1169 - val_gender_accuracy: 0.5232
